In [16]:
import numpy as np
import re

### Funkcja generująca tablicę numpy z wystarczającą liczbą wierszy dla każdego ograniczenia, funkcją celu i wystarczającą liczbą kolumn dla zmiennych, zmiennych luzu, M (maks./min.) i odpowiadającej im wartości.
variables - liczba zmiennych x1, x2... 

constraints - liczba ograniczeń x1 < x2 ...

In [17]:
def generate_matrix(variables , constraints):    
    table = np.zeros((constraints + 1, variables + constraints + 2))    
    return table

print(generate_matrix(5, 5))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


**next_round_right** - sprawdza czy wymagana jest 1+ oś obrotu z powodu ujemnego elementu w najdalszej prawej kolumnie, wyłączając dolną wartość.

**next_round_bottom** - sprawdza czy wymagana jest 1+ oś obrotu z powodu ujemnego elementu w dolnym wierszu, wyłączając wartość końcową.

table - wygenerowana w punkcie pierwszym tablica

In [18]:
def next_round_right(table):    
    m = min(table[:-1,-1])    
    if m>= 0:        
        return False    
    else:        
        return True

def next_round_bottom(table):    
    lr = len(table[:,0])   
    m = min(table[lr-1,:-1])    
    if m>=0:
        return False
    else:
        return True

### Niezależnie od tego, czy wymagane są dodatkowe elementy obrotowe, musimy określić, gdzie znajdują się te elementy.
**find_negative_right** - znajduje elementy negatywne w skrajnej prawej kolumnie.

**find_negative_bottom** - znajduje elementy negatywne w dolnym rzędzie

In [19]:
def find_negative_right(table):
    lc = len(table[0,:])
    m = min(table[:-1,lc-1])
    if m<=0:        
        n = np.where(table[:-1,lc-1] == m)[0][0]
    else:
        n = None
    return n

def find_negative_bottom(table):
    lr = len(table[:,0])
    m = min(table[lr-1,:-1])
    if m<=0:
        n = np.where(table[lr-1,:-1] == m)[0][0]
    else:
        n = None
    return n

### Znalezienie elementu obrodu odpowiadającego wartościom które znaleźliśmy w poprzednim punkcie

**local_pivot_right** - znalezienie elementu obrotu dla negatywnych elementów w ostatniej kolumnie

**local_pivot_bottom** - znalezienie elementu obrotu dla negatywnych elementów w dolnym rzędzie 

In [20]:
def local_pivot_right(table):
    total = []        
    r = find_negative_right(table)
    row = table[r,:-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1,c]
    for i, b in zip(col,table[:-1,-1]):
        if i**2>0 and b/i>0:
            total.append(b/i)
        else:                
            total.append(10000)
    index = total.index(min(total))        
    return [index,c]

def local_pivot_bottom(table):
    if next_round_bottom(table):
        total = []
        n = find_negative_bottom(table)
        for i,b in zip(table[:-1,n],table[:-1,-1]):
            if b/i >0 and i**2>0:
                total.append(b/i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index,n]

### Funkcja przestawna, obraca się wokół elementu, aby usunąć negatywny wpis w ostatniej kolumnie lub wierszu i zwraca zaktualizowaną tabelę.

In [21]:
def pivot(row, col, table):
    lr = len(table[:,0])
    lc = len(table[0,:])
    t = np.zeros((lr,lc))
    pr = table[row,:]
    if table[row,col]**2>0:
        e = 1/table[row,col]
        r = pr*e
        for i in range(len(table[:,col])):
            k = table[i,:]
            c = table[i,col]
            if list(k) == list(pr):
                continue
            else:
                t[i,:] = list(k-r*c)
        t[row,:] = list(r)
        return t
    else:
        raise Exception('Cannot pivot on this element.')

### Funkcja pozwalająca na zamiane ciągu na zmienne float, 
Wszystkie zmienne x1, x2, xn  muszą być podane w kolejności, i wszystkie muszą być wpisane,  np x2 >=1  to będzie 0x1 + x2 + 0x3 >=1, spacje nie mają znaczenia.

In [22]:
def convert(eq):
    if re.match(r'(.*)(<=|>=)(.*)', eq):
        left = re.match(r'(.*)(<=|>=)(.*)', eq).group(1).replace(" ", "").replace("+", " +").replace("-", " -").lower()
        side = re.match(r'(.*)(<=|>=)(.*)', eq).group(2).replace(" ", "")
        right = re.match(r'(.*)(<=|>=)(.*)', eq).group(3).replace(" ", "")

        eq = []
        left = re.split(' ', left)
        if(left[0] == ''):
            left.pop(0)

        for x in left:
            x = re.match(r'^(-?\+?[0-9]*)x[0-9]*', x).group(1)
            if(x == '' or x == '+'):
                x = 1
            elif(x == '-'):
                x = -1
            eq.append(float(x))
            
        if(side == '<='):
            eq = [float(i) for i in eq]
            eq.append(float(right))
        elif(side == '>='):
            eq = [float(i)*-1 for i in eq]
            eq.append(float(right)*-1)
        
        
        return eq
    else: 
        raise Exception('Wrong input constrain.')

### Funkcja konwertująca współczynniki funkcji celu na odpowiednie dla problemów minimalizacyjncych
W przypadku problemów maksymalizacyjnych, współczynniki w ostatnim wierszu tabeli Simpleksowej powstają poprzez obliczenie odwrotności współczynników funkcji celu. W przypadku problemów minimalizacyjncych natomiast, potrzebujemy aby współczynniki odpowiadały funkcji celu, dlatego definiujemy pomocniczą funkcję, która je odpowiednio zamienia.

In [23]:
def convert_min(table):
    table[-1,:-2] = [-1*i for i in table[-1,:-2]]
    table[-1,-1] = -1*table[-1,-1]    
    return table

### Funkcją generująca odpowiedną liczbę zmiennych x1, x2 itd.


In [24]:
def generate_variables(table):
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    v = []
    for i in range(var):
        v.append('x'+str(i+1))
    return v

### Funkcja sprawdzająca czy do macierzy można dodać kolejne ograniczenia

In [25]:
def add_cons(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i,:]:                       
            total += j**2
        if total == 0: 
            empty.append(total)
    if len(empty)>1:
        return True
    else:
        return False

### Funkcja wstawiająca odpowiednie dane z warunków do tablicy wygenerowanej wcześniej

In [26]:
def constraint(table, eq):
    if add_cons(table) == True:
        lc = len(table[0,:])
        lr = len(table[:,0])
        var = lc - lr -1      
        j = 0
        while j < lr:            
            row_check = table[j,:]
            total = 0
            for i in row_check:
                total += float(i**2)
            if total == 0:                
                row = row_check
                break
            j +=1
        eq = convert(eq)
        i = 0
        while i<len(eq)-1:
            row[i] = eq[i]
            i +=1        
        row[-1] = eq[-1]
        row[var+j] = 1    
    else:
        raise Exception('Cannot add another constraint.')

### Sprawdzenie czy można dodać funkcje celu do tablicy
Można dodać funkcję celu jedynie wtedy, gdy są zdefiniowane wszystkie ograniczenie - t.j. wtedy gdy zostaje jedynie ostatni rząd tabeli Simpleksowej z zerami. W tym celu sumujemy kwadraty poszczególnych współczynników w każdym z rzędów i liczymy rząd jako pusty gdy suma ta wynosi 0.

In [27]:
def add_obj(table):
    lr = len(table[:,0])
    empty = []
    for i in range(lr):
        total = 0        
        for j in table[i,:]:
            total += j**2
        if total == 0:
            empty.append(total)    
    if len(empty)==1:
        return True
    else:
        return False

### Dodanie funkcji celu do tablicy

In [28]:
def convert2(eq):
    if re.match(r'(.*)(=)(.*)', eq):
        right = re.match(r'.*=(.*)', eq).group(1).replace(" ", "").replace("+", " +").replace("-", " -").lower()
        eq = []
        right = re.split(' ', right)
        if(right[0] == ''):
            right.pop(0)

        for x in right:
            x = re.match(r'^(-?\+?[0-9]*)x[0-9]*', x).group(1)
            if(x == '' or x == '+'):
                x = 1
            elif(x == '-'):
                x = -1
            eq.append(float(x))
        eq.append(float(0))
        return eq
    
def obj(table,eq):
    if add_obj(table)==True:
        eq = convert2(eq)
        lr = len(table[:,0])
        row = table[lr-1,:]
        i = 0        
        while i<len(eq)-1:
            row[i] = eq[i]*-1
            i +=1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        raise Exception('You must finish adding constraints before the objective function can be added.')

### Złączenie wszystkich elementów składowych w jeden algorytm
Dwie pętle while do określenia, czy wymagany jest element obrotu 1+, jeżeli tak następuje jego zlokalizowanie, obrót wokół niego i kontynuowania procesu, aż wszystkie elementy ujemne zostaną usunięte z ostatniej kolumny i wiersza.  Następnie zostaną wygenerowane zmienne dla x1 do xn i przypisane wartości zgodnie z ich pozycjami w tabeli. Dodatkowo max lub min otrzyma odpowiednią wartość.

In [29]:
def calculate(table, flag='max'):
    if(flag=='min'):
        table = convert_min(table)
        
    while next_round_right(table)==True:
        table = pivot(local_pivot_right(table)[0],local_pivot_right(table)[1],table)
    while next_round_bottom(table)==True:
        table = pivot(local_pivot_bottom(table)[0],local_pivot_bottom(table)[1],table)        
    lc = len(table[0,:])
    lr = len(table[:,0])
    var = lc - lr -1
    i = 0
    val = {}
    for i in range(var):
        col = table[:,i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]            
            val[generate_variables(table)[i]] = table[loc,-1]
        else:
            val[generate_variable(table)[i]] = 0
    if(flag == 'min'):
        val['min'] = table[-1,-1]*-1
    elif(flag== 'max'):
        val['max'] = table[-1,-1]
    return val

In [30]:
    # Testy czy działa
    m = generate_matrix(3,3)
    constraint(m,'x1 + x2 +3x3 <= 30')
    constraint(m,'x1 + 0x2 + x3 >= 5')
    constraint(m,'x1 + x2 + 0x3 <= 45')
    obj(m,'P = x1 -3x2 + 3x3')
    print(calculate(m, 'min'))
    print(m)

{'x1': 5.0, 'x2': 25.0, 'x3': 70.0, 'min': -70.0}
[[ 1.  1.  3.  1.  0.  0.  0. 30.]
 [-1. -0. -1.  0.  1.  0.  0. -5.]
 [ 1.  1.  0.  0.  0.  1.  0. 45.]
 [ 1. -3.  3. -0. -0. -0.  1. -0.]]
<ipython-input-20-9385b0969448>:21: RuntimeWarning: divide by zero encountered in double_scalars
  if b/i >0 and i**2>0:
